Responses have been categorized into various multi-labels, but are not easy to comprehend. Therefore labels will be formulated into simple format.

Note: This notebook have lot of manual interventionist steps

# Packages Installation

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=24bd9ac4d2025c1b39db54409f9b9ecc6af56a190d3bc18afad4878a196e7260
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


# Read Data

In [ ]:
import pandas as pd
path_to_step1 = "/content/drive/MyDrive/Colab Notebooks/CPU/Step1/"
cwiki_responses = pd.read_json(path_to_step1+"output/cwiki_responses.json")

In [ ]:
# split the responses to process separately
cw_data = cwiki_responses[cwiki_responses.Source == 'CW']
iki_data = cwiki_responses[cwiki_responses.Source == 'IKITracs']
cw_data = cw_data.reset_index(drop=True)
print('CW data', len(cw_data))
iki_data = iki_data.reset_index(drop=True)
print('IKI data', len(iki_data))

CW data 46973
IKI data 4087


# CW

In [ ]:
# Extract Information from CWInfo, as we need these columns to work upon to
# identify labels to apply to the Response
import string
cw_data['OverviewCategory'],cw_data['QuestionText'],cw_data['QuestionDefinition'], \
            cw_data['QuestionCode'] = zip(*cw_data.apply(lambda x:
                                        (x['CWInfo']['OverviewCategory'],
                                         x['CWInfo']['QuestionText'],
                                         x['CWInfo']['QuestionDefinition'],
                                         x['CWInfo']['QuestionCode']),
                                               axis=1))
cwiki_responses['ResponseText'] = cwiki_responses.ResponseText.str.strip()
cw_data['OverviewCategory'] = cw_data.OverviewCategory.str.strip()
cw_data['QuestionText'] = cw_data.QuestionText.str.strip()
cw_data['QuestionDefinition'] = cw_data.QuestionDefinition.str.strip()

# dropping cost related data and NA for QuestionText
cw_data = cw_data[~cw_data["QuestionText"].str.lower().str.contains("cost", na=False)]
cw_data = cw_data[cw_data["ResponseText"] != '"']
cw_data = cw_data[cw_data["QuestionText"].notna()]
cw_data = cw_data.reset_index(drop=True)

## Labels Criterea

In this subsection we will define the criterea of evaluating if we should attach the True/False flag to the ResponseText for the particular Label class.

In [ ]:
temp = cw_data[['OverviewCategory','QuestionText','QuestionDefinition','ResponseText']]
temp.groupby(['OverviewCategory','QuestionText','QuestionDefinition']).ResponseText.size()

OverviewCategory              QuestionText                                            QuestionDefinition                                                            
Adaptation Commitments        Action and priority                                     Adaptation action/priority                                                        10031
                              Adapt Now sector                                        GCA_Sector                                                                         2107
                              Adapt Now sector (2)                                    GCA_Sector_2                                                                        776
                              Adapt Now sector (3)                                    GCA_Sector_3                                                                         97
                              Adapt Now subsector                                     GCA_subsector                                        

Target

In [ ]:
# identifying the criterea for 'Target' label selection
temp = cw_data[['OverviewCategory','QuestionText','QuestionDefinition','ResponseText']]
temp = temp.reset_index(drop =True)
# combining the text from three columns
temp['col text'] = temp.apply(lambda x: str(x['OverviewCategory']) + " "+
                                 str(x['QuestionText']) + " " +
                              str(x['QuestionDefinition']),axis=1)
# search for word Target in combined text
temp['target'] = temp.apply(lambda x: True if 'target'  in x['col text'].lower()
                             else False, axis=1)
temp[temp['target'] == True].groupby(['OverviewCategory']).QuestionText.value_counts()

OverviewCategory              QuestionText                           
Adaptation Commitments        Targets                                    1900
                              Sectoral Target                             193
Sectoral Mitigation Actions   Conditional part of mitigation target       609
                              Unconditional part of mitigation target     347
Sectoral Mitigation Plans     Conditional part of mitigation target       406
                              Unconditional part of mitigation target     263
Sectoral Mitigation Policies  Conditional part of mitigation target       100
                              Unconditional part of mitigation target      55
Sectoral Mitigation Targets   Sectoral targets on                        2968
                              Conditional part of mitigation target       897
                              Unconditional part of mitigation target     473
                              Emission reduction potential              

In [ ]:
# Definining Target based on 'QuestionText', Based on above analysis
targets_type_QuestionText = ['Sectoral Target',
                              'Targets',
                              'Sectoral targets on',
                              'Conditional part of mitigation target',
                              'Unconditional part of mitigation target',
                              'Tracking progress towards targets']

Action

In [ ]:
# # identifying the criterea for 'Action' label selection
temp['action'] = temp.apply(lambda x: True if 'action'  in x['col text'].lower()
                             else False, axis=1)
print(len(temp[temp['action'] ==True]))
temp[temp['action'] == True].groupby(['OverviewCategory']).QuestionText.value_counts()

16428


OverviewCategory             QuestionText                                          
Adaptation Commitments       Action and priority                                       10031
                             Sectoral Conditional Actions                               1655
                             Sectoral Unconditional Actions                              556
                             Implementing Agency for Sectoral Conditonal Actions          70
                             Funders for Sectoral Unconditional Actions                   56
                             Implementing Agency for Sectoral Unconditional Actions       39
                             Funders for Sectoral Conditional Actions                     21
Sectoral Mitigation Actions  Downstream actions                                         2385
                             Conditional part of mitigation target                       609
                             Emission reduction potential                      

In [ ]:
# Definining Action based on 'QuestionText'. Based on above analysis
actions_type_QuestionText = ['Sectoral Unconditional Actions',
                             'Sectoral Conditional Actions',
                             'Action and priority',
                             'Building on existing downstream actions',
                             'Downstream actions']

Policies and Plans

In [ ]:
# # identifying the criterea for 'Policies and Plans' label selection
temp['policies_plans'] = temp.apply(lambda x: True if ('policy' in
                                                  x['col text'].lower() or 'plan'
                                                   in x['col text'].lower())
                                                    else False, axis=1)
temp[temp['policies_plans'] == True].groupby(['OverviewCategory']).QuestionText.\
                                                                value_counts()

OverviewCategory              QuestionText                           
Adaptation Commitments        Sectoral Policies                           202
Sectoral Mitigation Plans     Sectoral plans on                          4124
                              Conditional part of mitigation target       406
                              Unconditional part of mitigation target     263
                              Emission reduction potential                249
                              Technology needs                            159
                              Capacity building needs                     103
Sectoral Mitigation Policies  Building on existing NAMAs                  144
                              Conditional part of mitigation target       100
                              Capacity building needs                      64
                              Unconditional part of mitigation target      55
                              Technology needs                          

In [ ]:
# define policy based questiontext. Based on above analysis
policies_plans_type_QuestionText = ['Sectoral Policies',
                                    'Sectoral plans on',
                                    'Building on existing NAMAs',
                                    'Existing sectoral policies',
                                    'Upstream policies on']

Conditionality

In [ ]:
# # identifying the criterea for 'Conditional' label selection
temp['conditional'] = temp.apply(lambda x: True if 'conditional'  in x['col text'].lower()
                             else False, axis=1)
print(len(temp[temp['conditional'] ==True]))
temp[temp['conditional'] == True].groupby(['OverviewCategory']).QuestionText.value_counts()

5547


OverviewCategory              QuestionText                                          
Adaptation Commitments        Sectoral Conditional Actions                              1655
                              Sectoral Unconditional Actions                             556
                              Implementing Agency for Sectoral Conditonal Actions         70
                              Funders for Sectoral Unconditional Actions                  56
                              Implementing Agency for Sectoral Unconditional Actions      39
                              Funders for Sectoral Conditional Actions                    21
Sectoral Mitigation Actions   Conditional part of mitigation target                      609
                              Unconditional part of mitigation target                    347
Sectoral Mitigation Plans     Conditional part of mitigation target                      406
                              Unconditional part of mitigation target         

In [ ]:
# Definining Conditional based on 'QuestionText'
conditional_type_QuestionText = ['Sectoral Conditional Actions',
       'Implementing Agency for Sectoral Conditonal Actions',
       'Funders for Sectoral Conditional Actions',
       'Conditional part of mitigation target']
unconditional_type_QuestionText = ['Sectoral Unconditional Actions',
       'Implementing Agency for Sectoral Unconditional Actions',
       'Funders for Sectoral Unconditional Actions',
       'Unconditional part of mitigation target']


## Response Merge

In [ ]:
# getting rid of columns which we added for label extraction
cw_data.drop(columns = ['OverviewCategory', 'QuestionText',
       'QuestionDefinition', 'IkiInfo','QuestionCode'], inplace=True)

In [ ]:
# dropping some garbage responses values
garbage_values = ['(18)','" (p. 29)','” (p. 7)', '" (21)', '" (17)', '” (12)',
                  '" (6)','" (9)','” (p. 8)','" (14)','" (43)','” (p. 18)',
                  'other','" (20)','" (7)','" (16)','" (9-10)','” (p. 12)','” (17)',
                  'etc."', '"(10)', 'In french','" (p. 8)','"" (12)', '" (10-11)',
                  '” (p. 11)', '” (p. 4)', 'etc." (5)','" (18)','" (19)','” (p. 5)',
                  '" (p. 31', '” (p. 17)','" (12)', '" (10)', '(19)','" (38)',
                  '" (22)','" (11)','” (11)','” (22)','" (p. 6)','" (4)','" (p. 22)',
                  '" (33)','and "','and" (4)','” (p. 6)','" (p. 33)','" (p. 10)',
                  '" (45)','” (p. 10)','” (16)','" (44)','” (p. 19)','" (5)',
                  '" (13)',',','and." (20)','" (p. 11)','(26)','" (6-7)','” (21)',
                  'In French','" (20-21)','" (p. 32)','" (p. 7)','” (p. 35)',
                  '" (p. 3)','" (42)', '” (p. 25)','"(p. 30)','" (p. 31)','No',
                  'Yes','” (p. 24)','" (41)','" (15)','(17)','(18-19)','" (23)',
                  '” (p. 9)','” (p. 23)','" (p. 9)','" (p. 30)',
                  'added entry for map','" (conditional)', '" (unconditional)',
                  'y, but Not Available','" (p. 30)','" (p. 9)','"ongoing"',
                  'and " (21)']

cw_data['check'] = cw_data.ResponseText.apply(lambda x: True if x in
                                                      garbage_values else False)
cw_data = cw_data[cw_data.check == False].reset_index(drop =True)
cw_data.drop(columns= ['check'], inplace=True)
print(" Rows Count after dropping garbage values",len(cw_data))

 Rows Count after dropping garbage values 42346


In [ ]:
print("Row Count",len(cw_data))
cw_responses = cw_data.groupby(['ResponseText','Alpha3','Country','Document'], as_index = False)\
                    .agg({'CWInfo':list, 'Source':set,'ResponseWordcount':set})

# Setting the Source = 'CW'
cw_responses['Source'] = 'CW'
cw_responses['ResponseWordcount'] = cw_responses.ResponseWordcount.apply(lambda x: list(x)[0])
print("Rows count after Merge:",len(cw_responses))

Row Count 42346
Rows count after Merge: 26861


In [ ]:
cw_responses['QuestionTextList'] = cw_responses.CWInfo.apply(lambda x:
                                                [i['QuestionText'] for i in x])
cw_responses['OverviewCategoryList'] = cw_responses.CWInfo.apply(lambda x:
                                            [i['OverviewCategory'] for i in x])

## Labels Extraction

In [ ]:
def labels_extraction(df:pd.Series,check_in):
    """
    We will use the defined question Categories to check for if True/False should
    be assigned to class label.

    Params
    -------------
    df: series
    check_in: QuestionText List to check in for

    Return
    ----------
    df: Series with flag True or False

    """

    return df.apply(lambda x: any([False if i is None else True if i in check_in else False for i in x]))

In [ ]:
cw_responses['Target'] = labels_extraction(cw_responses.QuestionTextList, targets_type_QuestionText)
print('Value Counts for Target')
print(cw_responses.Target.value_counts(),'\n')
cw_responses['Action'] = labels_extraction(cw_responses.QuestionTextList, actions_type_QuestionText)
print('Value Counts for Action')
print(cw_responses.Action.value_counts(),'\n')
cw_responses['Policies_Plans'] = labels_extraction(cw_responses.QuestionTextList, policies_plans_type_QuestionText)
print('Value Counts for Policies and Plans')
print(cw_responses.Policies_Plans.value_counts(),'\n')
cw_responses['Conditional'] = labels_extraction(cw_responses.QuestionTextList, conditional_type_QuestionText)
print('Value Counts for Conditional')
print(cw_responses.Conditional.value_counts(),'\n')
cw_responses['Unconditional'] = labels_extraction(cw_responses.QuestionTextList, unconditional_type_QuestionText)
print('Value Counts for Unconditional')
print(cw_responses.Unconditional.value_counts(),'\n')

Value Counts for Target
False    20859
True      6002
Name: Target, dtype: int64 

Value Counts for Action
False    15396
True     11465
Name: Action, dtype: int64 

Value Counts for Policies and Plans
False    21303
True      5558
Name: Policies_Plans, dtype: int64 

Value Counts for Conditional
False    23493
True      3368
Name: Conditional, dtype: int64 

Value Counts for Unconditional
False    25300
True      1561
Name: Unconditional, dtype: int64 



In [ ]:
cw_responses['Adaptation'] = cw_responses.OverviewCategoryList.apply(lambda x:
                                                  True if 'adaptation' in str(x).lower()
                                                  else False)
print("Value Counts for Adaptation")
print(cw_responses.Adaptation.value_counts(),'\n')

cw_responses['Mitigation'] = cw_responses.OverviewCategoryList.apply(lambda x:
                                                  True if 'mitigation' in str(x).lower()
                                                  else False)

print("Value Counts for Mitigation")
print(cw_responses.Mitigation.value_counts(),'\n')

Value Counts for Adaptation
True     14746
False    12115
Name: Adaptation, dtype: int64 

Value Counts for Mitigation
False    14723
True     12138
Name: Mitigation, dtype: int64 



In [ ]:
# def label_transform(df, labels):
#     """
#     In the dataset the same response can appear multiple times depending on
#     'QuestionText'.
#     Ex: ResponseText = 'Increase contribution of Wind Energy by 30% by 2030'
#     being considered for label = Target.
#     This response can appear at multiple places depending on QuestionText it is
#     answering. Lets say the Response appeared thrice
#                QuestionText = ['Conditional Target','Time Frame', 'Sectoral Target']
#     The way the Response Text has been labelled it will appear first time as True
#     and second time as False, and third time as True becuase the Label is being
#     decided based on QuestionText. So now once we group the Responses, for
#     'Target' class it will have a list [True, False, True]. So now we need to
#     give final Label for Target class. For this, if ResponseText has appeared even
#     once as True for the class we say its True, else its False.

#     Params
#     -------------
#     df: dataframe fo responses
#     labels: list or str of the columns (Class) to be looked at

#     Return
#     ------------
#     df: dataframe

#     """
#     # we count how many times the Response appear as True for label and how many
#     # times it appear as False.
#     if type(labels) == str:
#       df[labels] = df[labels].apply(lambda x: True if True in x else False)
#     elif type(labels) == list:
#       for label in labels:
#           df[label] = df[label].apply(lambda x: True if True in x else False)
#     return df

# def boolvalidate(data, col):
#     print('Total count', len(data))
#     print('{} count'.format(col), sum(data[col] == True),
#           'Non {} count'.format(col), sum(data[col]==False))
#     print('Missig values', sum(data[col].isna()))

In [ ]:
# checking which Question categories still left
# As can be seen from Question text we ignore these category to be inlcuded in these labels
list_ = set(temp.QuestionText.unique())
remaining_ques = list_ - (set(targets_type_QuestionText).union(actions_type_QuestionText,policies_plans_type_QuestionText))
remaining_ques

{'Adapt Now sector',
 'Adapt Now sector (2)',
 'Adapt Now sector (3)',
 'Adapt Now subsector',
 'Adapt Now subsector (2)',
 'Adapt Now subsector (3)',
 'Capacity Building Needs for Sectoral Implementation',
 'Capacity building needs',
 'Emission reduction potential',
 'Funders for Sectoral Conditional Actions',
 'Funders for Sectoral Unconditional Actions',
 'Implementing Agency for Sectoral Conditonal Actions',
 'Implementing Agency for Sectoral Unconditional Actions',
 'Technology Transfer Needs for Sectoral Implementation',
 'Technology needs',
 'Time frame',
 'Tracking Progress for Sectoral Implementation'}

# IKI

In [ ]:
# extracting the relevant information from IKIInfo, to define the labelling criterea
iki_data['Sheet'] = iki_data.IkiInfo.apply(lambda x: x['IkiSheet'])
iki_data['Parameter'] = iki_data.IkiInfo.apply(lambda x: x['parameter'])

## Labels Criterea

In [ ]:
# defining the labelling criterea for categories
netzero_ikisheet = ['netzero']
target_ikisheet = ['netzero', 'targets']
ghg_yes_ikiparameter = ['T_Transport_Unc','T_Transport_C']
ghg_no_ikiparameter = ['T_Adaptation_Unc', 'T_Adaptation_C', 'T_Transport_O_Unc',
                                      'T_Transport_O_C']

# mitigation and adaptation willneed both Sheet info and Paramter info
mitigation_ikiparameter = ['T_Transport_O_Unc',
                           'T_Transport_Unc',
                           'T_Transport_C'
                           'T_Transport_O_C']
mitigation_ikisheet = ['mitigation']
adaptation_ikiparameter = ['T_Adaptation_Unc', 'T_Adaptation_C']
adaptation_ikisheet = ['adaptation']


Conditionality

In [ ]:
temp = iki_data[['Sheet','Parameter']]

In [ ]:
# identifying the criterea for Conditionality
temp['unc'] = temp.Parameter.apply(lambda x: False if x is None else x.lower().endswith('_unc'))
print(temp[temp.unc==True].Parameter.unique())

temp['con'] = temp.Parameter.apply(lambda x: False if x is None else x.lower().endswith('_c'))
print(temp[temp.con==True].Parameter.unique())

['T_Transport_O_Unc' 'T_Economy_Unc' 'T_Transport_Unc' 'T_Adaptation_Unc'
 'M_Volume_unc']
['T_Netzero_C' 'T_Economy_C' 'T_Transport_O_C' 'T_Adaptation_C'
 'T_Transport_C' 'M_Volume_c']


<ipython-input-62-c90ac5489d57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['unc'] = temp.Parameter.apply(lambda x: False if x is None else x.lower().endswith('_unc'))
<ipython-input-62-c90ac5489d57>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['con'] = temp.Parameter.apply(lambda x: False if x is None else x.lower().endswith('_c'))


In [ ]:
conditional_ikiparameter = ['T_Netzero_C',
                            'T_Economy_C',
                            'T_Transport_O_C',
                            'T_Adaptation_C',
                            'T_Transport_C',
                            'M_Volume_c']
unconditional_ikiparameter = ['T_Transport_O_Unc',
                              'T_Economy_Unc',
                              'T_Transport_Unc',
                              'T_Adaptation_Unc',
                              'M_Volume_unc']

## Response Merge

In [ ]:
# getting rid of columns which we added for label extraction
iki_data.drop(columns = ['Sheet','Parameter'], inplace=True)


# dropping some garbage responses values
garbage_values = ['(18)','" (p. 29)','” (p. 7)', '" (21)', '" (17)', '” (12)',
                  '" (6)','" (9)','” (p. 8)','" (14)','" (43)','” (p. 18)',
                  'other','" (20)','" (7)','" (16)','" (9-10)','” (p. 12)','” (17)',
                  'etc."', '"(10)', 'In french','" (p. 8)','"" (12)', '" (10-11)',
                  '” (p. 11)', '” (p. 4)', 'etc." (5)','" (18)','" (19)','” (p. 5)',
                  '" (p. 31', '” (p. 17)','" (12)', '" (10)', '(19)','" (38)',
                  '" (22)','" (11)','” (11)','” (22)','" (p. 6)','" (4)','" (p. 22)',
                  '" (33)','and "','and" (4)','” (p. 6)','" (p. 33)','" (p. 10)',
                  '" (45)','” (p. 10)','” (16)','" (44)','” (p. 19)','" (5)',
                  '" (13)',',','and." (20)','" (p. 11)','(26)','" (6-7)','” (21)',
                  'In French','" (20-21)','" (p. 32)','" (p. 7)','” (p. 35)',
                  '" (p. 3)','" (42)', '” (p. 25)','"(p. 30)','" (p. 31)','No',
                  'Yes','” (p. 24)','" (41)','" (15)','(17)','(18-19)','" (23)',
                  '” (p. 9)','” (p. 23)','" (p. 9)','" (p. 30)',
                  'added entry for map' ]

iki_data['check'] = iki_data.ResponseText.apply(lambda x: True if x in
                                                      garbage_values else False)
iki_data = iki_data[iki_data.check == False].reset_index(drop =True)
iki_data.drop(columns= ['check'], inplace=True)
print(" Rows Count after dropping garbage values",len(iki_data))

 Rows Count after dropping garbage values 4076


In [ ]:
print("Row Count",len(iki_data))
iki_responses = iki_data.groupby(['ResponseText','Alpha3','Country','Document'], as_index = False)\
                    .agg({'IkiInfo':list, 'Source':set,'ResponseWordcount':set})

# Setting the Source = 'IKITracs'
iki_responses['Source'] = 'IKITracs'
iki_responses['ResponseWordcount'] = iki_responses.ResponseWordcount.apply(lambda x: list(x)[0])
print("Rows count after Merge:",len(iki_responses))

Row Count 4076
Rows count after Merge: 3579


In [ ]:
iki_responses['IKISheetList'] = iki_responses.IkiInfo.apply(lambda x:
                                                    [i['IkiSheet'] for i in x])
iki_responses['ParameterList'] = iki_responses.IkiInfo.apply(lambda x:
                                                   [i['parameter'] for i in x])

## Labels Extraction

In [ ]:
# Getting Target labels based on Sheet Info
iki_responses['Target'] = labels_extraction(iki_responses.IKISheetList, target_ikisheet)
print('Value Counts for Target')
print(iki_responses.Target.value_counts(),'\n')

# Getting Target labels based on Sheet Info
iki_responses['Netzero'] = labels_extraction(iki_responses.IKISheetList, netzero_ikisheet)
print('Value Counts for Netzero')
print(iki_responses.Netzero.value_counts(),'\n')

# Getting Adaptation labels based on Sheet and Parameter Info
iki_responses['Adaptation_sheet'] = labels_extraction(iki_responses.IKISheetList,
                                                          adaptation_ikisheet)
iki_responses['Adaptation_param'] = labels_extraction(iki_responses.ParameterList,
                                                      adaptation_ikiparameter)

iki_responses["Adaptation"] = iki_responses.apply(lambda x: True if (x['Adaptation_sheet']
                                                                     or
                                                                     x['Adaptation_param'])
                                                                  else False,axis=1)
iki_responses.drop(columns  = ['Adaptation_sheet','Adaptation_param'], inplace=True)
print('Value Counts for Adaptation')
print(iki_responses.Adaptation.value_counts(),'\n')

# Getting Adaptation labels based on Sheet and Parameter Info
iki_responses['Mitigation_sheet'] = labels_extraction(iki_responses.IKISheetList, mitigation_ikisheet)

iki_responses['Mitigation_param'] = labels_extraction(iki_responses.ParameterList,
                                                      mitigation_ikiparameter)

iki_responses["Mitigation"] = iki_responses.apply(lambda x: True if (x['Mitigation_sheet']
                                                                     or
                                                                     x['Mitigation_param'])
                                                                  else False,axis=1)
iki_responses.drop(columns  = ['Mitigation_sheet','Mitigation_param'], inplace=True)
print('Value Counts for Mitigation')
print(iki_responses.Mitigation.value_counts(),'\n')


iki_responses['GHG'] = labels_extraction(iki_responses.ParameterList, ghg_yes_ikiparameter)
iki_responses['NonGHG'] = labels_extraction(iki_responses.ParameterList, ghg_no_ikiparameter)
iki_responses['GHG'] = iki_responses.apply(lambda x: True if x['GHG'] else False
                                           if x['NonGHG'] else None, axis=1)
iki_responses.drop(columns = ['NonGHG'], inplace=True)
print('Value Counts for GHG')
print(iki_responses.GHG.value_counts(),'\n')

Value Counts for Target
False    2730
True      849
Name: Target, dtype: int64 

Value Counts for Netzero
False    3516
True       63
Name: Netzero, dtype: int64 

Value Counts for Adaptation
False    3306
True      273
Name: Adaptation, dtype: int64 

Value Counts for Mitigation
True     2222
False    1357
Name: Mitigation, dtype: int64 

Value Counts for GHG
False    223
True      66
Name: GHG, dtype: int64 



In [ ]:
iki_responses['Conditional'] = labels_extraction(iki_responses.ParameterList, conditional_ikiparameter)
print('Value Counts for Conditional')
print(iki_responses.Conditional.value_counts(),'\n')
iki_responses['Unconditional'] = labels_extraction(iki_responses.ParameterList, unconditional_ikiparameter)
print('Value Counts for UnConditional')
print(iki_responses.Unconditional.value_counts(),'\n')

Value Counts for Conditional
False    3311
True      268
Name: Conditional, dtype: int64 

Value Counts for UnConditional
False    3080
True      499
Name: Unconditional, dtype: int64 



# Merge Datasets
In this subsection we merge both the responses dataset, perform basic checks and cleaning. Further we de-normalize the dataset by doing the groupby over ResponseText. In addition we perform the language detection for the response text. ( In CW data language is uniformly 'english', however thats not the case for IKI Tracs)

In [ ]:
cw_responses.drop(columns  = ['QuestionTextList','OverviewCategoryList'], inplace=True)
iki_responses.drop(columns  = ['IKISheetList', 'ParameterList'], inplace=True)

In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
import re
import langdetect
from tqdm import tqdm
tqdm.pandas()

# we will need it for langauge detection.
# while langdetect from google works great on longer text, however since the
# model behind is not determinsitic therefore for small text it cannot be relied upon.
model_name = 'qanastek/51-languages-classifier'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=512, truncation=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)


def contains_digits_and_punctuation_only(string):
    # adapted from Chatgpt: to check if string contains only digits and punct
    pattern = r'^[0-9!"#$%&\'()*+,\-.\/:;<=>?@\[\\\]^_`{|}~]+$'
    return re.match(pattern, string) is not None

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
# identify language of response, for if soemthing is non-english
iki_responses['language'] = iki_responses.progress_apply(lambda x: classifier(x['ResponseText'])
                                if x['ResponseWordcount'] > 2 else None, axis=1)


path_to_step2 = '/content/drive/MyDrive/Colab Notebooks/CPU/Step2/'
pdf_para = pd.read_json(path_to_step2 + 'output/pdf_para.json')
html_para = pd.read_json(path_to_step2 + 'output/html_para.json')

iki_responses['language'] = iki_responses.language.apply(lambda x: x[0]['label'][:2]
                                                                if x else None)
language_list = set(iki_responses.language.unique())
doc_language = set(pdf_para.language.unique()).union(set(html_para.language.unique()))
others = language_list - doc_language - {None}

# some manual corrections
other1 = {'zh', 'ar'}
iki_responses['language'] = iki_responses.language.apply(lambda x: 'en' if x in
                                                                 other1 else x)
iki_responses['language'] = iki_responses.language.apply(lambda x: 'en' if x in
                                                                 others else x)

100%|██████████| 3579/3579 [13:01<00:00,  4.58it/s]


In [ ]:
iki_responses = iki_responses[iki_responses.language.notna()]
cw_responses['language'] = 'en'

In [ ]:
# combine both dataset (Cw and IKI)
responses = pd.concat([iki_responses,cw_responses], ignore_index = True)

In [ ]:
# changing the document names to align with Step 2 nomenclature.
responses.Document.replace({'1st NDC': 'First NDC', '2nd NDC':'Second NDC',
                            'Archived 2nd NDC':'Archived Second NDC',
                            '2nd NDC Supplement':'NDC reference document',
                            'Archived NDC 1.1':'Archived Revised First NDC',
                            'Archived Updated NDC':'Archived Revised First NDC',
                            'LTS 1.0':'LTS', 'NDC 1.1': 'Revised First NDC',
                            'Updated NDC':'Revised First NDC',
                            'Revised First NDC (archived)':'Archived Revised First NDC',
                'Climate Change Response (Zero Carbon) Amendment Act 2019':'Climate Act'},
                           inplace=True)

In [ ]:
len(responses)

30404

In [ ]:
#save the results
import json
path_to_step3 = "/content/drive/MyDrive/Colab Notebooks/CPU/Step3/"
jsonfile = responses.to_json(orient="records")
parsed = json.loads(jsonfile)
with open(path_to_step3 +'output/responses_grouped.json', 'w') as file:
    json.dump(parsed, file, indent=4)